<a href="https://colab.research.google.com/github/apester/IME/blob/main/Lab19_Improved_LSTM_GRU_Industrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚧 Industrial Predictive Maintenance with LSTM and GRU (Improved Version)

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout
import warnings
warnings.filterwarnings('ignore')

## Simulated Industrial Dataset

In [ ]:
np.random.seed(42)
n = 5000
timestamps = np.arange(n)
sensor1 = np.sin(timestamps / 40) + np.random.normal(0, 0.1, n)
sensor2 = np.cos(timestamps / 50) + np.random.normal(0, 0.1, n)
failure = (timestamps % 400 < 10).astype(int)

df = pd.DataFrame({
    'sensor1': sensor1,
    'sensor2': sensor2,
    'failure': failure
})

## Preprocessing

In [ ]:
scaler = StandardScaler()
df[['sensor1', 'sensor2']] = scaler.fit_transform(df[['sensor1', 'sensor2']])

def sequences(data, steps=30):
    X, y = [], []
    for i in range(len(data)-steps):
        X.append(data[['sensor1', 'sensor2']].iloc[i:i+steps].values)
        y.append(data['failure'].iloc[i+steps])
    return np.array(X), np.array(y)

X, y = sequences(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## LSTM Model Definition (Improved)

In [ ]:
lstm_model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8468 - loss: 0.4274 - val_accuracy: 0.9724 - val_loss: 0.1262
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9774 - loss: 0.1134 - val_accuracy: 0.9724 - val_loss: 0.1264
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9752 - loss: 0.1178 - val_accuracy: 0.9724 - val_loss: 0.1268
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9766 - loss: 0.1138 - val_accuracy: 0.9724 - val_loss: 0.1273
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9678 - loss: 0.1438 - val_accuracy: 0.9724 - val_loss: 0.1307


## GRU Model Definition (Improved)

In [ ]:
gru_model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    GRU(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8275 - loss: 0.5005 - val_accuracy: 0.9724 - val_loss: 0.1269
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9763 - loss: 0.1164 - val_accuracy: 0.9724 - val_loss: 0.1281
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9745 - loss: 0.1244 - val_accuracy: 0.9724 - val_loss: 0.1281
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9735 - loss: 0.1224 - val_accuracy: 0.9724 - val_loss: 0.1299
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9745 - loss: 0.1207 - val_accuracy: 0.9724 - val_loss: 0.1289


## Evaluation with Zero Division Handling

In [ ]:
y_pred_lstm = (lstm_model.predict(X_test) > 0.5).astype(int)
print('LSTM Evaluation:\n', classification_report(y_test, y_pred_lstm, zero_division=0))

y_pred_gru = (gru_model.predict(X_test) > 0.5).astype(int)
print('GRU Evaluation:\n', classification_report(y_test, y_pred_gru, zero_division=0))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
LSTM Evaluation:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       972
           1       0.00      0.00      0.00        22

    accuracy                           0.98       994
   macro avg       0.49      0.50      0.49       994
weighted avg       0.96      0.98      0.97       994

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
GRU Evaluation:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       972
           1       0.00      0.00      0.00        22

    accuracy                           0.98       994
   macro avg       0.49      0.50      0.49       994
weighted avg       0.96      0.98      0.97       994

